# 📊 Análise Exploratória de Dados - Criptomoedas BTC/ETH

## 🎯 Objetivo
Este notebook realiza uma análise exploratória completa dos dados de criptomoedas Bitcoin (BTC) e Ethereum (ETH).

## 📋 Conteúdo
1. **Setup e Importação** - Bibliotecas necessárias
2. **Carregamento de Dados** - Dataset de criptomoedas
3. **Exploração Inicial** - Estrutura e limpeza dos dados
4. **Análises Temporais** - Evolução de preços e volumes
5. **Métricas Financeiras** - Retornos, volatilidade, drawdowns
6. **Visualizações Interativas** - Gráficos com Plotly
7. **Comparação BTC vs ETH** - Análises comparativas
8. **Conclusões** - Insights principais

---
**Data:** 20 de setembro de 2025  
**Autor:** Caio Vasconcelos  
**Fonte dos dados:** Dataset Kaggle - Historical Cryptocurrency Data

In [ ]:
# 📚 Importação de Bibliotecas
# ======================================

# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Análises estatísticas e científicas
from scipy import stats
from scipy.signal import find_peaks
# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✅ Todas as bibliotecas foram importadas com sucesso!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"📈 Plotly version: {px.__version__}")

## 📂 Carregamento e Preparação dos Dados

Carregaremos o dataset de criptomoedas e realizaremos a limpeza inicial dos dados, seguindo a mesma lógica do dashboard Streamlit.

In [ ]:
# 🔍 Carregamento do Dataset
# ================================

# Caminho para o arquivo de dados
DATA_PATH = "../data/cryptocurrency.csv"

# Carregar dados
try:
    df = pd.read_csv(DATA_PATH)
    print("✅ Dataset carregado com sucesso!")
    print(f"📊 Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
    print(f"📅 Período: {df['Date'].min()} até {df['Date'].max()}")
except FileNotFoundError:
    print("❌ Arquivo não encontrado. Verifique o caminho:", DATA_PATH)
    raise

# Visualizar primeiras linhas
print("\n🔎 Primeiras 5 linhas do dataset:")
df.head()

## 🔍 Exploração e Limpeza dos Dados

Vamos verificar a qualidade dos dados, identificar valores ausentes, duplicatas e preparar os dados para análise.

In [ ]:
# 📊 Qualidade dos Dados
# ========================

print("🔍 ANÁLISE DE QUALIDADE DOS DADOS")
print("=" * 40)

# 1. Valores ausentes
print("\n1️⃣ Valores Ausentes:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_summary = pd.DataFrame({
    'Ausentes': missing_data,
    'Percentual (%)': missing_percent.round(2)
})
print(missing_summary[missing_summary['Ausentes'] > 0])

# 2. Duplicatas
duplicates = df.duplicated().sum()
print(f"\n2️⃣ Duplicatas encontradas: {duplicates}")

# 3. Tipos de dados
print("\n3️⃣ Tipos de Dados:")
print(df.dtypes)

# 4. Estatísticas básicas
print("\n4️⃣ Estatísticas Descritivas:")
print(df.describe())

# 5. Moedas disponíveis
print("\n5️⃣ Moedas Disponíveis:")
print(df['Symbol'].value_counts())

print("\n✅ Análise de qualidade concluída!")

## 📅 Preparação Temporal dos Dados

Converteremos as datas e criaremos colunas temporais para facilitar as análises de séries temporais.

In [ ]:
# 🕒 Preparação Temporal
# =======================

# Converter coluna Date para datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Verificar se há datas inválidas
invalid_dates = df['Date'].isnull().sum()
if invalid_dates > 0:
    print(f"⚠️ Encontradas {invalid_dates} datas inválidas. Removendo...")
    df = df.dropna(subset=['Date'])

# Criar colunas temporais
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.day_name()
df['Quarter'] = df['Date'].dt.quarter

# Ordenar por data
df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)

print("✅ Preparação temporal concluída!")
print(f"📅 Período analisado: {df['Date'].min()} até {df['Date'].max()}")
print(f"📊 Total de registros após limpeza: {len(df)}")

# Visualizar estrutura temporal
print("\n📈 Distribuição por ano:")
print(df.groupby('Year').size())

## 📊 Análise das Moedas Disponíveis

Vamos analisar os períodos disponíveis para cada criptomoeda e determinar o período comum para comparações justas.

In [ ]:
# 💰 Análise das Moedas
# =====================

# Períodos por moeda
periodos = df.groupby('Symbol')['Date'].agg(['min', 'max', 'count'])
periodos.columns = ['Data Inicial', 'Data Final', 'Total Dias']
print("📅 Períodos disponíveis por moeda:")
print(periodos)

# Encontrar período comum (interseção)
start_date = df.groupby("Symbol")["Date"].min().max()  # Maior data inicial
end_date = df.groupby("Symbol")["Date"].max().min()    # Menor data final

print(f"\n🎯 Período comum identificado:")
print(f"📅 De: {start_date.date()}")
print(f"📅 Até: {end_date.date()}")

# Filtrar dados para período comum
df_common = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)].copy()
print(f"\n📊 Registros no período comum: {len(df_common)}")

# Estatísticas por moeda no período comum
stats_por_moeda = df_common.groupby('Symbol').agg({
    'Close': ['mean', 'std', 'min', 'max'],
    'Volume': ['mean', 'sum'],
    'Marketcap': ['mean', 'max']
}).round(2)

print("\n📈 Estatísticas por moeda (período comum):")
print(stats_por_moeda)

## 🎯 Definição do Período de Análise

Seguindo a mesma lógica do dashboard, definiremos o período comum entre BTC e ETH para uma comparação justa.

In [ ]:
# 🎯 Período de Análise Comum
# ============================

# Filtrar apenas BTC e ETH
crypto_focus = ['BTC', 'ETH']
df_focus = df[df['Symbol'].isin(crypto_focus)].copy()

# Recalcular período comum apenas para BTC/ETH
start_common = df_focus.groupby("Symbol")["Date"].min().max()
end_common = df_focus.groupby("Symbol")["Date"].max().min()

# Filtrar dataset para período comum
df_2015 = df_focus[(df_focus["Date"] >= start_common) & (df_focus["Date"] <= end_common)].copy()

print("🎯 ANÁLISE FOCADA EM BTC E ETH")
print("=" * 35)
print(f"📅 Período comum: {start_common.date()} até {end_common.date()}")
print(f"📊 Dias totais: {(end_common - start_common).days}")

# Estatísticas do período
stats_periodo = {
    'BTC': len(df_2015[df_2015['Symbol'] == 'BTC']),
    'ETH': len(df_2015[df_2015['Symbol'] == 'ETH']),
    'Total': len(df_2015)
}

print(f"\n📈 Estatísticas do período:")
for moeda, count in stats_periodo.items():
    print(f"   {moeda}: {count} registros")

# Reset index
df_2015 = df_2015.reset_index(drop=True)
df_2015["SNo"] = df_2015.index + 1

print("\n✅ Dataset preparado para análise comparativa BTC vs ETH!")

### Análises Iniciais encontradas no dataset:

##### É possível logo inicialmente criar a coluna ['Return'] onde se dá pelo calculo da variação percentual do preço de fechamento de um dia para o outro, ou seja, cada linha de df['Return'] mostra quanto (%) o preço de fechamento mudou em relação ao dia anterior.

In [ ]:
# Retorno Diário
df['Return'] = df['Close'].pct_change() * 100

In [ ]:
df

##### O primeiro index vai estar NaN pois não é possível calcular a partir do primeiro dia, somente a partir da variação da coluna ['Close']

In [ ]:
df

### Removendo NaN da Coluna 'Return'

In [ ]:
df = df.dropna(subset=['Return']).reset_index(drop=True)

In [ ]:
df.head(3)

### Evolução temporal do preço (gráfico de linha com Close).

In [ ]:
# Selecionar apenas dados do Bitcoin
btc = df[df['Symbol'] == 'BTC'].copy()

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(btc['Date'], btc['Close'], label='Fechamento', color='blue')
ax.set_xlabel('Data')
ax.set_ylabel('Preço (USD)')
ax.set_title('Evolução do Preço de Fechamento - Bitcoin')
ax.legend()
plt.show()

### Ciclos sazonais (ex: meses com maior valorização ou maior volume).

In [ ]:
# Ciclos sazonais apenas para Bitcoin - CORRIGIDO (apenas anos completos)
btc_completo = btc[btc['Year'] <= 2020].copy()
btc_monthly = btc_completo.groupby('Month').agg({'Return': 'mean', 'Volume': 'mean'}).reset_index()

fig, ax1 = plt.subplots(figsize=(10, 5))
color = 'tab:blue'
ax1.bar(btc_monthly['Month'], btc_monthly['Return'], color=color, alpha=0.6)
ax1.set_xlabel('Mês')
ax1.set_ylabel('Retorno Médio (%)', color=color)
ax1.set_title('Ciclos Sazonais - Bitcoin (2013-2020)')
ax1.set_xticks(range(1, 13))

ax2 = ax1.twinx()
color = 'tab:orange'
ax2.plot(btc_monthly['Month'], btc_monthly['Volume'], color=color, marker='o')
ax2.set_ylabel('Volume Médio', color=color)

# Adicionar nota sobre a correção
plt.figtext(0.5, 0.02, 'Nota: Análise usando apenas anos completos (2013-2020)', 
           ha='center', fontsize=8, style='italic')
plt.show()

#### *Janeiro: Pode ser um mês de maior atividade (talvez "efeito ano novo" nos investimentos)*
#### *Novembro: Pode ser um mês de maior valorização (talvez relacionado ao final do ano fiscal ou expectativas de fim de ano)*

### Períodos de alta/baixa do mercado.

In [ ]:
# Selecionar apenas dados do Bitcoin
btc = df[df['Symbol'] == 'BTC'].copy()

# CORREÇÃO: Usar apenas anos completos (2013-2020) para análises justas
print("=== CORREÇÃO DE DADOS ===")
print(f"Dataset original: {btc['Date'].min()} a {btc['Date'].max()}")
print(f"2021 tem apenas {len(btc[btc['Year'] == 2021])} dias (incompleto)")

# Criar dataset com anos completos para análises
btc_completo = btc[btc['Year'] <= 2020].copy()
print(f"Dataset corrigido: {btc_completo['Date'].min()} a {btc_completo['Date'].max()}")
print(f"Usando {len(btc_completo)} registros de anos completos")

# Calcular média móvel de 200 dias para identificar tendências
btc['MA200'] = btc['Close'].rolling(200).mean()

# Criar coluna para identificar se está acima ou abaixo da média móvel
btc['Trend'] = btc['Close'] > btc['MA200']

# Plotar preço com média móvel
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(btc['Date'], btc['Close'], label='Preço de Fechamento', alpha=0.7)
ax.plot(btc['Date'], btc['MA200'], label='Média Móvel 200 dias', color='red')
ax.fill_between(btc['Date'], btc['Close'].min(), btc['Close'].max(), 
                where=btc['Trend'], alpha=0.3, color='green', label='Período de Alta')
ax.set_xlabel('Data')
ax.set_ylabel('Preço (USD)')
ax.set_title('Períodos de Alta/Baixa do Mercado - Bitcoin')
ax.legend()
plt.show()

# Calcular drawdown (queda do pico)
btc['Peak'] = btc['Close'].cummax()
btc['Drawdown'] = (btc['Close'] - btc['Peak']) / btc['Peak'] * 100

fig, ax = plt.subplots(figsize=(14, 6))
ax.fill_between(btc['Date'], btc['Drawdown'], 0, alpha=0.3, color='red')
ax.plot(btc['Date'], btc['Drawdown'], color='red')
ax.set_xlabel('Data')
ax.set_ylabel('Drawdown (%)')
ax.set_title('Drawdowns do Bitcoin (Períodos de Queda)')
plt.show()

### Picos Históricos Reais

In [ ]:
# Identificar onde estão os picos históricos
picos_historicos = btc[btc['Close'] == btc['Peak']]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(btc['Date'], btc['Close'], label='Preço de Fechamento', alpha=0.7)
ax.plot(btc['Date'], btc['MA200'], label='Média Móvel 200 dias', color='red')

# Marcar os picos históricos
ax.scatter(picos_historicos['Date'], picos_historicos['Close'], 
           color='orange', s=50, label='Picos Históricos', zorder=5)

ax.set_xlabel('Data')
ax.set_ylabel('Preço (USD)')
ax.set_title('Preço do Bitcoin com Picos Históricos Marcados')
ax.legend()
plt.show()

### Comparação por ano (2013–2021): qual ano teve maior crescimento?

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
btc_annual['annual_return_%'].plot(kind='bar', ax=ax, color='skyblue')
ax.set_xlabel('Ano')
ax.set_ylabel('Retorno Anual (%)')
ax.set_title('Retorno Anual do Bitcoin (2013–2021)')
ax.set_xticklabels(btc_annual.index, rotation=0)
plt.tight_layout()
plt.show()

### Volatilidade diária: diferença entre High - Low.

In [ ]:
# Primeiro, criar a coluna de volatilidade diária
btc['DailyVolatility'] = btc['High'] - btc['Low']

# CORREÇÃO: Usar apenas anos completos para análise de volatilidade
btc_completo = btc[btc['Year'] <= 2020].copy()

# Análise de volatilidade por ano e mês - CORRIGIDO
btc_completo['Year_Month'] = btc_completo['Date'].dt.to_period('M')
btc_volatility_yearly_monthly = btc_completo.groupby(['Year', 'Month'])['DailyVolatility'].mean().reset_index()

# Top 10 períodos mais voláteis
top_volatile = btc_volatility_yearly_monthly.sort_values('DailyVolatility', ascending=False).head(10)

print("Top 10 períodos mais voláteis (Ano-Mês) - Anos Completos:")
for idx, row in top_volatile.iterrows():
    print(f"{int(row['Year'])}-{int(row['Month']):02d}: ${row['DailyVolatility']:.2f}")

# Heatmap de volatilidade por ano e mês - CORRIGIDO
pivot_volatility = btc_completo.pivot_table(values='DailyVolatility', index='Year', columns='Month', aggfunc='mean')

fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(pivot_volatility, annot=True, fmt='.0f', cmap='Reds', ax=ax)
ax.set_title('Heatmap de Volatilidade por Ano e Mês - Bitcoin (2013-2020)')
ax.set_xlabel('Mês')
ax.set_ylabel('Ano')
plt.figtext(0.5, 0.02, 'Nota: 2021 excluído por ter dados incompletos', 
           ha='center', fontsize=8, style='italic')
plt.show()

## Outliers Encontrados

In [ ]:
# CORREÇÃO: Usar apenas anos completos para análise de outliers
btc_completo = btc[btc['Year'] <= 2020].copy()

# Verificar outliers na volatilidade diária
Q1 = btc_completo['DailyVolatility'].quantile(0.25)
Q3 = btc_completo['DailyVolatility'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Identificar outliers
outliers = btc_completo[(btc_completo['DailyVolatility'] < limite_inferior) | (btc_completo['DailyVolatility'] > limite_superior)]

print(f"Total de outliers (2013-2020): {len(outliers)}")
print(f"Limite superior para outliers: ${limite_superior:.2f}")
print(f"Maior volatilidade registrada: ${btc_completo['DailyVolatility'].max():.2f}")

# 1. Boxplot para visualizar outliers
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Boxplot
ax1.boxplot(btc_completo['DailyVolatility'], vert=True)
ax1.set_ylabel('Volatilidade Diária (USD)')
ax1.set_title('Boxplot - Outliers na Volatilidade Diária (2013-2020)')

# 2. Scatter plot temporal dos outliers
ax2.scatter(btc_completo['Date'], btc_completo['DailyVolatility'], alpha=0.5, s=10, label='Normal')
ax2.scatter(outliers['Date'], outliers['DailyVolatility'], 
           color='red', s=30, label='Outliers', alpha=0.8)
ax2.axhline(y=limite_superior, color='red', linestyle='--', alpha=0.7, label='Limite Superior')
ax2.set_xlabel('Data')
ax2.set_ylabel('Volatilidade Diária (USD)')
ax2.set_title('Outliers ao Longo do Tempo (2013-2020)')
ax2.legend()

plt.tight_layout()
plt.show()

# 3. Top 10 dias mais voláteis (outliers extremos)
top_outliers = outliers.nlargest(10, 'DailyVolatility')[['Date', 'DailyVolatility', 'High', 'Low', 'Close']]
print("\nTop 10 outliers mais extremos (2013-2020):")
print(top_outliers)

### Tendência: Close - Open → positivo (valorizou), negativo (desvalorizou).

In [ ]:
# Primeiro, criar a coluna DailyTrend
btc['DailyTrend'] = np.where(btc['Close'] > btc['Open'], 1, 
                            np.where(btc['Close'] < btc['Open'], -1, 0))

# CORREÇÃO: Usar apenas anos completos
btc_completo = btc[btc['Year'] <= 2020].copy()

# Contagem mensal de dias de alta, baixa e estáveis
trend_counts = btc_completo.groupby(['Year', 'Month'])['DailyTrend'].value_counts().unstack(fill_value=0)

# Filtrar apenas alguns anos para ficar legível - CORRIGIDO
trend_recent = trend_counts.loc[
    (trend_counts.index.get_level_values(0) >= 2018) & 
    (trend_counts.index.get_level_values(0) <= 2020)
]

# Gráfico de barras empilhadas
trend_recent.index = trend_recent.index.map(lambda x: f"{x[0]}-{x[1]:02d}")
trend_recent.plot(kind='bar', stacked=True, figsize=(16, 6), 
                 color={1: 'green', -1: 'red', 0: 'gray'})
plt.xlabel('Ano-Mês')
plt.ylabel('Número de Dias')
plt.title('Tendência Diária do Bitcoin por Mês (2018-2020)')
plt.legend(['Baixa', 'Estável', 'Alta'])
plt.xticks(rotation=45)
plt.figtext(0.5, 0.02, 'Nota: Usando apenas anos completos', 
           ha='center', fontsize=8, style='italic')
plt.tight_layout()
plt.show()

### Comparação entre moedas: qual é mais volátil, qual valorizou mais.

In [ ]:
# CORREÇÃO: Usar df_2015 mas excluir 2021 incompleto
btc_eth = df_2015[df_2015['Year'] <= 2020].copy()

# Calcular volatilidade diária para ambas as moedas
btc_eth['DailyVolatility'] = btc_eth['High'] - btc_eth['Low']

# Calcular retorno diário para ambas as moedas
btc_eth['Return'] = btc_eth.groupby('Symbol')['Close'].pct_change() * 100

# 1. Comparação de Volatilidade Média
volatility_comparison = btc_eth.groupby('Symbol')['DailyVolatility'].agg(['mean', 'std', 'median']).reset_index()

print("Comparação de Volatilidade (2015-2020):")
print(volatility_comparison)

# 2. Comparação de Retorno Total
# Calcular retorno acumulado desde o início
returns_comparison = btc_eth.groupby('Symbol').apply(
    lambda x: ((x['Close'].iloc[-1] / x['Close'].iloc[0]) - 1) * 100
).reset_index(name='RetornoTotal_%')

print("\nComparação de Retorno Total (2015-2020):")
print(returns_comparison)

# 3. Gráficos de Comparação
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Volatilidade Média
ax1.bar(volatility_comparison['Symbol'], volatility_comparison['mean'], 
        color=['orange', 'blue'], alpha=0.7)
ax1.set_title('Volatilidade Média Diária (2015-2020)')
ax1.set_ylabel('Volatilidade (USD)')
for i, v in enumerate(volatility_comparison['mean']):
    ax1.text(i, v + v*0.01, f'${v:.2f}', ha='center', va='bottom')

# Gráfico 2: Retorno Total
ax2.bar(returns_comparison['Symbol'], returns_comparison['RetornoTotal_%'], 
        color=['orange', 'blue'], alpha=0.7)
ax2.set_title('Retorno Total (2015-2020)')
ax2.set_ylabel('Retorno (%)')
for i, v in enumerate(returns_comparison['RetornoTotal_%']):
    ax2.text(i, v + v*0.01, f'{v:.0f}%', ha='center', va='bottom')

# Gráfico 3: Evolução do Preço (Normalizado)
for symbol in ['BTC', 'ETH']:
    data = btc_eth[btc_eth['Symbol'] == symbol].copy()
    data['PriceNormalized'] = (data['Close'] / data['Close'].iloc[0]) * 100
    ax3.plot(data['Date'], data['PriceNormalized'], label=symbol, linewidth=2)
ax3.set_title('Evolução do Preço - Base 100 (2015-2020)')
ax3.set_ylabel('Preço Normalizado')
ax3.legend()

# Gráfico 4: Boxplot de Volatilidade
btc_vol = btc_eth[btc_eth['Symbol'] == 'BTC']['DailyVolatility']
eth_vol = btc_eth[btc_eth['Symbol'] == 'ETH']['DailyVolatility']
ax4.boxplot([btc_vol, eth_vol], labels=['BTC', 'ETH'])
ax4.set_title('Distribuição da Volatilidade Diária (2015-2020)')
ax4.set_ylabel('Volatilidade (USD)')

plt.figtext(0.5, 0.02, 'Nota: Análise usando apenas anos completos (2015-2020)', 
           ha='center', fontsize=8, style='italic')
plt.tight_layout()
plt.show()

# 4. Análise detalhada por ano - CORRIGIDO
yearly_analysis = btc_eth.groupby(['Symbol', 'Year']).agg({
    'Return': 'mean',
    'DailyVolatility': 'mean'
}).reset_index()

print("\nAnálise por Ano (2015-2020):")
pivot_return = yearly_analysis.pivot(index='Year', columns='Symbol', values='Return')
pivot_volatility = yearly_analysis.pivot(index='Year', columns='Symbol', values='DailyVolatility')

print("\nRetorno Médio por Ano (%):")
print(pivot_return.round(2))
print("\nVolatilidade Média por Ano (USD):")
print(pivot_volatility.round(2))

### Dias/meses com maior movimentação (picos de compra/venda).

In [ ]:
# CORREÇÃO: Usar apenas anos completos para análise de volume
btc_completo = btc[btc['Year'] <= 2020].copy()

# Análise de dias/meses com maior movimentação (volume de transações)

# 1. Identificar os dias com maior volume de transações
print("=== ANÁLISE DE VOLUME DE TRANSAÇÕES (2013-2020) ===")

# Top 10 dias com maior volume (apenas Bitcoin - anos completos)
top_volume_days = btc_completo.nlargest(10, 'Volume')[['Date', 'Volume', 'Close', 'DailyVolatility']]
print("\nTop 10 dias com maior volume de transações - Bitcoin (2013-2020):")
for idx, row in top_volume_days.iterrows():
    print(f"{row['Date'].strftime('%Y-%m-%d')}: ${row['Volume']:.2e} | Preço: ${row['Close']:.2f} | Volatilidade: ${row['DailyVolatility']:.2f}")

# 2. Análise mensal de volume médio - CORRIGIDO
monthly_volume = btc_completo.groupby(['Year', 'Month']).agg({
    'Volume': 'mean',
    'DailyVolatility': 'mean',
    'Return': 'mean'
}).reset_index()

# Top 10 meses com maior volume médio
top_volume_months = monthly_volume.nlargest(10, 'Volume')
print("\nTop 10 meses com maior volume médio - Bitcoin (2013-2020):")
for idx, row in top_volume_months.iterrows():
    print(f"{int(row['Year'])}-{int(row['Month']):02d}: ${row['Volume']:.2e} | Retorno médio: {row['Return']:.2f}%")

# 3. Correlação entre volume e movimentos de preço
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Volume ao longo do tempo - CORRIGIDO
ax1.plot(btc_completo['Date'], btc_completo['Volume'], alpha=0.7, linewidth=0.8)
ax1.set_title('Evolução do Volume de Transações - Bitcoin (2013-2020)')
ax1.set_ylabel('Volume')
ax1.set_xlabel('Data')

# Gráfico 2: Correlação Volume vs Volatilidade - CORRIGIDO
ax2.scatter(btc_completo['Volume'], btc_completo['DailyVolatility'], alpha=0.5, s=10)
ax2.set_xlabel('Volume')
ax2.set_ylabel('Volatilidade Diária (USD)')
ax2.set_title('Volume vs Volatilidade (2013-2020)')
correlation_vol_volatility = btc_completo['Volume'].corr(btc_completo['DailyVolatility'])
ax2.text(0.05, 0.95, f'Correlação: {correlation_vol_volatility:.3f}', 
         transform=ax2.transAxes, bbox=dict(boxstyle="round", facecolor='wheat'))

# Gráfico 3: Volume médio por mês do ano - CORRIGIDO
volume_by_month = btc_completo.groupby('Month')['Volume'].mean()
ax3.bar(range(1, 13), volume_by_month, alpha=0.7, color='lightblue')
ax3.set_xlabel('Mês')
ax3.set_ylabel('Volume Médio')
ax3.set_title('Volume Médio por Mês (2013-2020)')
ax3.set_xticks(range(1, 13))

# Gráfico 4: Heatmap de volume por ano e mês - CORRIGIDO
pivot_volume = btc_completo.pivot_table(values='Volume', index='Year', columns='Month', aggfunc='mean')
im = ax4.imshow(pivot_volume.values, cmap='Blues', aspect='auto')
ax4.set_title('Heatmap de Volume por Ano e Mês (2013-2020)')
ax4.set_xlabel('Mês')
ax4.set_ylabel('Ano')
ax4.set_xticks(range(12))
ax4.set_xticklabels(range(1, 13))
ax4.set_yticks(range(len(pivot_volume.index)))
ax4.set_yticklabels(pivot_volume.index)

plt.figtext(0.5, 0.02, 'Nota: 2021 excluído por ter dados incompletos', 
           ha='center', fontsize=8, style='italic')
plt.tight_layout()
plt.show()

# 4. Identificar períodos de maior atividade (volume + volatilidade) - CORRIGIDO
btc_completo['ActivityScore'] = (btc_completo['Volume'] / btc_completo['Volume'].mean()) * (btc_completo['DailyVolatility'] / btc_completo['DailyVolatility'].mean())
top_activity = btc_completo.nlargest(15, 'ActivityScore')[['Date', 'Volume', 'DailyVolatility', 'Return', 'Close', 'ActivityScore']]

print("\nTop 15 dias com maior atividade - volume + volatilidade (2013-2020):")
for idx, row in top_activity.iterrows():
    print(f"{row['Date'].strftime('%Y-%m-%d')}: Score: {row['ActivityScore']:.2f} | "
          f"Volume: ${row['Volume']:.2e} | Volatilidade: ${row['DailyVolatility']:.2f} | "
          f"Retorno: {row['Return']:.2f}%")

# 5. Análise de correlações - CORRIGIDO
print(f"\nCORRELAÇÕES (2013-2020):")
print(f"Volume vs Volatilidade: {btc_completo['Volume'].corr(btc_completo['DailyVolatility']):.3f}")
print(f"Volume vs |Retorno|: {btc_completo['Volume'].corr(abs(btc_completo['Return'])):.3f}")
print(f"Volume vs Preço: {btc_completo['Volume'].corr(btc_completo['Close']):.3f}")

### Gráficos de candlestick (com Plotly) para análise técnica.

### Correlação entre volume e preço (ex: volume alto → queda ou alta?).

### Identificar os maiores picos e quedas de preço.

### Relacionar com eventos históricos (ex: crash de 2018).
### Relacionar com eventos históricos (ex: queda do BTC em 2018, crash da pandemia em 2020).

### Comparar anos de maior valorização e maior queda.

### Identificar os piores crashes (retornos negativos extremos).

### Risco x Retorno (financeiro clássico)
##### Sharpe Ratio simplificado:
##### Sharpe = Retorno médio / Desvio-padrão


### Análise temporal de risco
##### Plotar volatilidade móvel (rolling std) para ver períodos de maior risco.
##### Cada valor em df["RollingVolatility"] mostra o quanto os retornos diários variaram (oscilaram) nos 30 dias anteriores àquela data. É uma medida de risco/instabilidade do ativo ao longo do tempo.

In [ ]:
df["RollingVolatility"] = df["Return"].rolling(30).std()

In [ ]:
# CORREÇÃO: Usar apenas anos completos
btc_completo = btc[btc['Year'] <= 2020].copy()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(btc_completo['Date'], btc_completo['RollingVolatility'])
ax.set_xlabel('Data')
ax.set_ylabel('Volatilidade Móvel')
ax.set_title('Volatilidade Móvel dos Retornos - Bitcoin (2013-2020)')
plt.figtext(0.5, 0.02, 'Nota: 2021 excluído por ter dados incompletos', 
           ha='center', fontsize=8, style='italic')
plt.show()

### Gráfico do preço de fechamento ao longo do tempo

In [ ]:
# Verificar até quando vai o dataset
print("=== VERIFICAÇÃO DO DATASET ===")
print(f"Data inicial: {btc['Date'].min()}")
print(f"Data final: {btc['Date'].max()}")

# Contar quantos dias temos por ano
days_per_year = btc.groupby('Year')['Date'].count()
print("\nDias por ano no dataset:")
for year, days in days_per_year.items():
    print(f"{year}: {days} dias")

# Verificar especificamente 2021
btc_2021 = btc[btc['Year'] == 2021]
print(f"\nEm 2021:")
print(f"Primeiro dia: {btc_2021['Date'].min()}")
print(f"Último dia: {btc_2021['Date'].max()}")
print(f"Total de dias em 2021: {len(btc_2021)}")
print(f"Meses presentes em 2021: {sorted(btc_2021['Month'].unique())}")

In [ ]:
# ANÁLISE FINAL CORRIGIDA
print("=== ANÁLISE CORRIGIDA - USANDO APENAS ANOS COMPLETOS ===")

# Comparar apenas anos completos (2013-2020)
btc_complete_years = btc[btc['Year'] <= 2020]

print(f"Dataset corrigido vai de {btc_complete_years['Date'].min().date()} até {btc_complete_years['Date'].max().date()}")
print(f"Total de registros: {len(btc_complete_years)} dias")

print("\nVolume médio anual (em bilhões USD):")
yearly_volume_complete = btc_complete_years.groupby('Year')['Volume'].mean()
for year, vol in yearly_volume_complete.items():
    print(f"{year}: ${vol/1e9:.1f} bilhões")

print("\nVolatilidade média anual:")
yearly_volatility_complete = btc_complete_years.groupby('Year')['DailyVolatility'].mean()
for year, vol in yearly_volatility_complete.items():
    print(f"{year}: ${vol:.2f}")

print("\nRetorno médio anual (%):")
yearly_return_complete = btc_complete_years.groupby('Year')['Return'].mean()
for year, ret in yearly_return_complete.items():
    print(f"{year}: {ret:.2f}%")

print("\n🎯 CORREÇÃO APLICADA COM SUCESSO!")
print("✅ Todas as análises agora usam apenas anos completos (2013-2020)")
print("✅ Dados de 2021 (incompletos) foram excluídos das análises comparativas")
print("✅ Gráficos e estatísticas agora são confiáveis e justos")

## 💹 Cálculo de Retornos e Métricas Financeiras

Vamos calcular os retornos diários e implementar as métricas do dashboard: drawdowns, Sharpe ratio, eficiência de recuperação, etc.

In [ ]:
# 💰 Cálculo de Retornos e Métricas
# =================================

# Calcular retornos diários por moeda
df_2015['Return'] = df_2015.groupby('Symbol')['Close'].pct_change() * 100

# Função para classificar retornos
def classify_return(return_value):
    if pd.isna(return_value):
        return 'Neutro'
    elif return_value > 0:
        return 'Positivo'
    elif return_value < 0:
        return 'Negativo'
    else:
        return 'Neutro'

# Aplicar classificação
df_2015['Return_Status'] = df_2015['Return'].apply(classify_return)

print("📊 MÉTRICAS FINANCEIRAS POR MOEDA")
print("=" * 40)

# Calcular métricas para cada moeda
for symbol in ['BTC', 'ETH']:
    data = df_2015[df_2015['Symbol'] == symbol].copy()

    print(f"\n🪙 {symbol}:")
    print("-" * 20)

    # 1. Valor médio
    valor_medio = data['Close'].mean()
    print(f"💰 Valor Médio: ${valor_medio:.2f}")

    # 2. Retorno médio diário
    retorno_medio = data['Return'].mean()
    print(f"📈 Retorno Médio Diário: {retorno_medio:.4f}%")

    # 3. Volatilidade (desvio padrão dos retornos)
    volatilidade = data['Return'].std()
    print(f"📊 Volatilidade: {volatilidade:.4f}%")

    # 4. Sharpe Ratio (retorno/volatilidade - assumindo rf=0)
    if volatilidade > 0:
        sharpe = retorno_medio / volatilidade
        print(f"🎯 Sharpe Ratio: {sharpe:.4f}")
    else:
        print("🎯 Sharpe Ratio: N/A")

    # 5. Drawdown máximo
    data['Cumulative_Return'] = (1 + data['Return'].fillna(0) / 100).cumprod()
    data['Peak'] = data['Cumulative_Return'].expanding().max()
    data['Drawdown'] = ((data['Cumulative_Return'] / data['Peak']) - 1) * 100
    max_drawdown = abs(data['Drawdown'].min())
    print(f"📉 Max Drawdown: {max_drawdown:.2f}%")

    # 6. Tendência de valorização (% dias positivos)
    status_counts = data['Return_Status'].value_counts()
    total_days = len(data)
    positive_pct = (status_counts.get('Positivo', 0) / total_days) * 100
    print(f"🚀 Dias Positivos: {positive_pct:.1f}%")

    # 7. Eficiência de recuperação
    recovery_times = []
    in_drawdown = False
    drawdown_start = 0

    for i, row in data.iterrows():
        if row['Drawdown'] < -5 and not in_drawdown:
            in_drawdown = True
            drawdown_start = i
        elif row['Drawdown'] >= -1 and in_drawdown:
            recovery_time = i - drawdown_start
            if recovery_time > 0:
                recovery_times.append(recovery_time)
            in_drawdown = False

    if recovery_times:
        avg_recovery_days = np.mean(recovery_times)
        efficiency_score = max(0, 100 - (avg_recovery_days / 365 * 100))
        print(f"🔄 Eficiência Recuperação: {efficiency_score:.1f}% ({avg_recovery_days:.1f} dias médios)")
    else:
        print("🔄 Eficiência Recuperação: Sem dados suficientes")

print("\n✅ Cálculo de métricas concluído!")

## 📈 Visualizações Interativas

Vamos criar gráficos interativos replicando as visualizações do dashboard Streamlit.

In [ ]:
# 📊 Gráfico 1: Preços Históricos com Picos - BTC
# ================================================

btc_data = df_2015[df_2015['Symbol'] == 'BTC'].copy()

# Encontrar picos históricos
prices = btc_data['Close'].values
peaks, _ = find_peaks(prices, height=prices.mean(), distance=30)

fig1 = go.Figure()

# Linha principal
fig1.add_trace(go.Scatter(
    x=btc_data['Date'],
    y=btc_data['Close'],
    mode='lines',
    name='BTC',
    line=dict(color='#f7931a', width=2),
    hovertemplate='<b>BTC</b><br>Data: %{x}<br>Preço: $%{y:,.0f}<extra></extra>'
))

# Marcar picos
if len(peaks) > 0:
    fig1.add_trace(go.Scatter(
        x=btc_data.iloc[peaks]['Date'],
        y=btc_data.iloc[peaks]['Close'],
        mode='markers',
        name='Picos BTC',
        marker=dict(color='red', size=8, symbol='triangle-up'),
        hovertemplate='<b>Pico BTC</b><br>Data: %{x}<br>Preço: $%{y:,.0f}<extra></extra>'
    ))

fig1.update_layout(
    title="Preços Históricos BTC com Picos (2015-2021)",
    xaxis_title="Data",
    yaxis_title="Preço BTC (USD)",
    height=500,
    hovermode='x unified',
    showlegend=True
)

fig1.show()

In [ ]:
# 📊 Gráfico 2: Volume de Transações - BTC
# ========================================

btc_data = df_2015[df_2015['Symbol'] == 'BTC'].copy()

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=btc_data['Date'],
    y=btc_data['Volume'],
    name='Volume BTC',
    marker_color='#f7931a',
    opacity=0.7,
    hovertemplate='<b>BTC Volume</b><br>Data: %{x}<br>Volume: %{y:,.0f}<extra></extra>'
))

fig2.update_layout(
    title="Volume de Transações BTC (2015-2021)",
    xaxis_title="Data",
    yaxis_title="Volume",
    height=400,
    hovermode='x unified',
    showlegend=False
)

fig2.show()

In [ ]:
# 📊 Gráfico 3: Comparação BTC vs ETH
# ====================================

btc_data = df_2015[df_2015['Symbol'] == 'BTC'].copy()
eth_data = df_2015[df_2015['Symbol'] == 'ETH'].copy()

# Calcular picos históricos
btc_data['Peak'] = btc_data['Close'].expanding().max()
btc_picos = btc_data[btc_data['Close'] == btc_data['Peak']]

eth_data['Peak'] = eth_data['Close'].expanding().max()
eth_picos = eth_data[eth_data['Close'] == eth_data['Peak']]

fig3 = go.Figure()

# BTC
fig3.add_trace(go.Scatter(
    x=btc_data['Date'],
    y=btc_data['Close'],
    mode='lines',
    name='BTC',
    line=dict(color='#f7931a', width=3),
    hovertemplate='<b>BTC</b><br>Data: %{x}<br>Preço: $%{y:,.0f}<extra></extra>'
))

# Picos BTC
fig3.add_trace(go.Scatter(
    x=btc_picos['Date'],
    y=btc_picos['Close'],
    mode='markers',
    name='Picos BTC',
    marker=dict(color='#ff6b35', size=8, symbol='triangle-up'),
    hovertemplate='<b>Pico BTC</b><br>Data: %{x}<br>Preço: $%{y:,.0f}<extra></extra>'
))

# ETH
fig3.add_trace(go.Scatter(
    x=eth_data['Date'],
    y=eth_data['Close'],
    mode='lines',
    name='ETH',
    line=dict(color='#627eea', width=3),
    hovertemplate='<b>ETH</b><br>Data: %{x}<br>Preço: $%{y:,.0f}<extra></extra>'
))

# Picos ETH
fig3.add_trace(go.Scatter(
    x=eth_picos['Date'],
    y=eth_picos['Close'],
    mode='markers',
    name='Picos ETH',
    marker=dict(color='#4a90e2', size=8, symbol='triangle-up'),
    hovertemplate='<b>Pico ETH</b><br>Data: %{x}<br>Preço: $%{y:,.0f}<extra></extra>'
))

fig3.update_layout(
    title="Comparação BTC vs ETH - Preços e Picos Históricos (2015-2021)",
    xaxis_title="Data",
    yaxis_title="Preço (USD)",
    height=500,
    hovermode='x unified',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
)

fig3.show()

## 📊 Análise Estatística Avançada

Vamos realizar análises estatísticas mais profundas dos retornos e correlações entre BTC e ETH.

In [ ]:
# 🔬 Análise Estatística dos Retornos
# ===================================

print("📊 ANÁLISE ESTATÍSTICA DOS RETORNOS")
print("=" * 40)

# Preparar dados de retornos
btc_returns = df_2015[df_2015['Symbol'] == 'BTC']['Return'].dropna()
eth_returns = df_2015[df_2015['Symbol'] == 'ETH']['Return'].dropna()

print("\n📈 Estatísticas Descritivas dos Retornos Diários:")
print("-" * 50)

stats_df = pd.DataFrame({
    'BTC': btc_returns.describe(),
    'ETH': eth_returns.describe()
})
print(stats_df.round(4))

# Testes de normalidade
print("\n🧪 Testes de Normalidade (Shapiro-Wilk):")
btc_shapiro = stats.shapiro(btc_returns)
eth_shapiro = stats.shapiro(eth_returns)

print(f"BTC - Estatística: {btc_shapiro.statistic:.4f}, p-valor: {btc_shapiro.pvalue:.4f}")
print(f"ETH - Estatística: {eth_shapiro.statistic:.4f}, p-valor: {eth_shapiro.pvalue:.4f}")

if btc_shapiro.pvalue < 0.05:
    print("BTC: Retornos NÃO seguem distribuição normal")
else:
    print("BTC: Retornos seguem distribuição normal")

if eth_shapiro.pvalue < 0.05:
    print("ETH: Retornos NÃO seguem distribuição normal")
else:
    print("ETH: Retornos seguem distribuição normal")

# Correlação entre BTC e ETH
correlation = btc_returns.corr(eth_returns)
print(f"\n🔗 Correlação entre retornos BTC e ETH: {correlation:.4f}")

# Interpretação da correlação
if correlation > 0.7:
    print("💪 Correlação FORTE positiva")
elif correlation > 0.3:
    print("🤝 Correlação MODERADA positiva")
elif correlation > -0.3:
    print("🤷 Correlação FRACA")
elif correlation > -0.7:
    print("⚠️ Correlação MODERADA negativa")
else:
    print("💔 Correlação FORTE negativa")